In [ ]:
# mount Google Drive

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# create a class directory for pedestrians

!mkdir "/content/drive/MyDrive/Dataset/HazyPerson/Images/pedestrian"

In [ ]:
# copy images to the new directory

!cp "/content/drive/MyDrive/Dataset/HazyPerson/Images/"* "/content/drive/MyDrive/Dataset/HazyPerson/Images/pedestrian/"

cp: -r not specified; omitting directory '/content/drive/MyDrive/Dataset/HazyPerson/Images/pedestrian'


In [ ]:
# rename images

import os
import shutil

directory = '/content/drive/MyDrive/Dataset/HazyPerson/Images/pedestrian/'
counter = 1

for filename in os.listdir(directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        new_filename = str(counter) + '.jpg'
        src = os.path.join(directory, filename)
        dst = os.path.join(directory, new_filename)
        shutil.move(src, dst)
        counter += 1

In [ ]:
# install necessary packages

!pip install torchvision
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.1 MB/s eta 0:00:00


In [ ]:
# clone DINO repository

!git clone https://github.com/RavindishaWi/dino.git
%cd dino

Cloning into 'dino'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 190 (delta 68), reused 61 (delta 58), pack-reused 106
Receiving objects: 100% (190/190), 24.45 MiB | 34.97 MiB/s, done.
Resolving deltas: 100% (121/121), done.
/content/dino


In [ ]:
# create a new folder 'ssl_finetuning' to save the trained weights
# download pretrained DINO ViT S/8 weights

!mkdir ssl_finetuning
!wget -O ssl_finetuning/full_checkpoint.pth https://dl.fbaipublicfiles.com/dino/dino_deitsmall8_pretrain/dino_deitsmall8_pretrain_full_checkpoint.pth

--2023-06-20 16:10:34--  https://dl.fbaipublicfiles.com/dino/dino_deitsmall8_pretrain/dino_deitsmall8_pretrain_full_checkpoint.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.8.35, 13.35.8.51, 13.35.8.19, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.8.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 352284107 (336M) [application/zip]
Saving to: ‘ssl_finetuning/full_checkpoint.pth’

ssl_finetuning/full 100%[===================>] 335.96M  5.18MB/s    in 79s     

2023-06-20 16:11:54 (4.23 MB/s) - ‘ssl_finetuning/full_checkpoint.pth’ saved [352284107/352284107]



In [ ]:
# train DINO for WiderPerson dataset

!python main_dino.py --arch vit_small --batch_size 32 --patch_size 8 --data_path /content/drive/MyDrive/Dataset/HazyPerson/Images --output_dir ssl_finetuning

Using cache found in /root/.cache/torch/hub/facebookresearch_xcit_main
Will run the code on one GPU.
| distributed init (rank 0): env://
git:
  sha: 7c446df5b9f45747937fb0d72314eb9f7b66930a, status: clean, branch: main

arch: vit_small
batch_size_per_gpu: 32
clip_grad: 3.0
data_path: /content/drive/MyDrive/Dataset/HazyPerson/Images
dist_url: env://
drop_path_rate: 0.1
epochs: 100
freeze_last_layer: 1
global_crops_scale: (0.4, 1.0)
gpu: 0
local_crops_number: 8
local_crops_scale: (0.05, 0.4)
local_rank: 0
lr: 0.0005
min_lr: 1e-06
momentum_teacher: 0.996
norm_last_layer: True
num_workers: 10
optimizer: adamw
out_dim: 65536
output_dir: ssl_finetuning
patch_size: 8
rank: 0
saveckp_freq: 20
seed: 0
teacher_temp: 0.04
use_bn_in_head: False
use_fp16: True
warmup_epochs: 10
warmup_teacher_temp: 0.04
warmup_teacher_temp_epochs: 0
weight_decay: 0.04
weight_decay_end: 0.4
world_size: 1
Data loaded: there are 1052 images.
Student and Teacher are built: they are both vit_small network.
Loss, optimiz

In [ ]:
!python visualize_attention.py --image_path /content/drive/MyDrive/Models/trained_weights/214.jpg --pretrained_weights /content/dino/ssl_finetuning/checkpoint.pth

In [ ]:
# move trained checkpoint weights to 'hazy_person_weights' directory

import shutil
import os

source_folder = '/content/dino/ssl_finetuning'
destination_folder = '/content/drive/MyDrive/Models/hazy_person_weights'
filename = 'checkpoint.pth'

source_path = os.path.join(source_folder, filename)
destination_path = os.path.join(destination_folder, filename)

shutil.move(source_path, destination_path)

'/content/drive/MyDrive/Models/hazy_person_weights/checkpoint.pth'

In [ ]:
!python main_dino.py --batch_size 32 --patch_size 8 --arch vit_small --data_path /content/drive/MyDrive/Dataset/HazyPerson/Images --output_dir /content/drive/MyDrive/Models/hazy_person_weights

Using cache found in /root/.cache/torch/hub/facebookresearch_xcit_main
Will run the code on one GPU.
| distributed init (rank 0): env://
git:
  sha: 7c446df5b9f45747937fb0d72314eb9f7b66930a, status: has uncommited changes, branch: main

arch: vit_small
batch_size_per_gpu: 32
clip_grad: 3.0
data_path: /content/drive/MyDrive/Dataset/HazyPerson/Images
dist_url: env://
drop_path_rate: 0.1
epochs: 300
freeze_last_layer: 1
global_crops_scale: (0.4, 1.0)
gpu: 0
local_crops_number: 8
local_crops_scale: (0.05, 0.4)
local_rank: 0
lr: 0.0005
min_lr: 1e-06
momentum_teacher: 0.996
norm_last_layer: True
num_workers: 10
optimizer: adamw
out_dim: 65536
output_dir: /content/drive/MyDrive/Models/hazy_person_weights
patch_size: 8
rank: 0
saveckp_freq: 20
seed: 0
teacher_temp: 0.04
use_bn_in_head: False
use_fp16: True
warmup_epochs: 10
warmup_teacher_temp: 0.04
warmup_teacher_temp_epochs: 0
weight_decay: 0.04
weight_decay_end: 0.4
world_size: 1
Data loaded: there are 1052 images.
Student and Teacher are b

In [ ]:
!python visualize_attention.py --image_path /content/drive/MyDrive/Models/trained_weights/AIR-1.jpg --pretrained_weights /content/drive/MyDrive/Models/hazy_person_weights/checkpoint.pth

Take key teacher in provided checkpoint dict
Pretrained weights found at /content/drive/MyDrive/Models/hazy_person_weights/checkpoint.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])
./attn-head0.png saved.
./attn-head1.png saved.
./attn-head2.png saved.
./attn-head3.png saved.
./attn-head4.png saved.
./attn-head5.png saved.
/content/dino/combined_attention.png saved.
Output image saved successfully.


In [ ]:
!python visualize_attention.py --image_path /content/drive/MyDrive/Models/trained_weights/950.jpg --pretrained_weights /content/drive/MyDrive/Models/hazy_person_weights/checkpoint.pth

Take key teacher in provided checkpoint dict
Pretrained weights found at /content/drive/MyDrive/Models/hazy_person_weights/checkpoint.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])
./attn-head0.png saved.
./attn-head1.png saved.
./attn-head2.png saved.
./attn-head3.png saved.
./attn-head4.png saved.
./attn-head5.png saved.
/content/dino/combined_attention.png saved.
gt_boxes: [(88, 1, 206, 294, 'person'), (204, 1, 297, 312, 'person')], pred_boxes: [[389, 327, 396, 351], [238, 322, 250, 333], [390, 298, 396, 310], [0, 257, 105, 351], [33, 234, 53, 252], [389, 164, 396, 252], [93, 0, 297, 310]]
gt_boxes: [(88, 1, 206, 294, 'person'), (204, 1, 297, 312, 'person')], pred_boxes: [[389, 327, 396, 351], [238, 322, 250, 333], [390, 298, 396, 310], [0, 257, 105, 351], [33, 234, 53, 252], [38

In [ ]:
!python visualize_attention.py --image_path /content/drive/MyDrive/Models/trained_weights/95.jpg --pretrained_weights /content/drive/MyDrive/Models/hazy_person_weights/checkpoint0200.pth

Take key teacher in provided checkpoint dict
Pretrained weights found at /content/drive/MyDrive/Models/hazy_person_weights/checkpoint0200.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])
./attn-head0.png saved.
./attn-head1.png saved.
./attn-head2.png saved.
./attn-head3.png saved.
./attn-head4.png saved.
./attn-head5.png saved.
/content/dino/combined_attention.png saved.
Output image saved successfully.


In [ ]:
!python visualize_attention.py --image_path /content/drive/MyDrive/FYP/test_set/testing/picture/018.jpg

Please use the `--pretrained_weights` argument to indicate the path of the checkpoint to evaluate.
Since no pretrained weights have been provided, we load the reference pretrained DINO weights.
./attn-head0.png saved.
./attn-head1.png saved.
./attn-head2.png saved.
./attn-head3.png saved.
./attn-head4.png saved.
./attn-head5.png saved.
/content/dino/combined_attention.png saved.
gt_boxes: [(88, 1, 206, 294, 'person'), (204, 1, 297, 312, 'person')], pred_boxes: [[71, 0, 83, 13], [55, 0, 69, 17], [0, 0, 51, 104]]
gt_boxes: [(88, 1, 206, 294, 'person'), (204, 1, 297, 312, 'person')], pred_boxes: [[71, 0, 83, 13], [55, 0, 69, 17], [0, 0, 51, 104]]
gt_boxes: [(88, 1, 206, 294, 'person'), (204, 1, 297, 312, 'person')], pred_boxes: [[71, 0, 83, 13], [55, 0, 69, 17], [0, 0, 51, 104]]
gt_boxes: [(88, 1, 206, 294, 'person'), (204, 1, 297, 312, 'person')], pred_boxes: [[71, 0, 83, 13], [55, 0, 69, 17], [0, 0, 51, 104]]
gt_boxes: [(88, 1, 206, 294, 'person'), (204, 1, 297, 312, 'person')], pred_bo